<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Init" data-toc-modified-id="Init-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Init</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Helper functions</a></span></li></ul></li><li><span><a href="#Descriptive-stats" data-toc-modified-id="Descriptive-stats-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Descriptive stats</a></span><ul class="toc-item"><li><span><a href="#Summary-table" data-toc-modified-id="Summary-table-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Summary table</a></span><ul class="toc-item"><li><span><a href="#Categorical-vars" data-toc-modified-id="Categorical-vars-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Categorical vars</a></span></li><li><span><a href="#Numeric-vars" data-toc-modified-id="Numeric-vars-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Numeric vars</a></span></li></ul></li><li><span><a href="#When-did-people-receive-ESPs?" data-toc-modified-id="When-did-people-receive-ESPs?-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>When did people receive ESPs?</a></span></li><li><span><a href="#How-many-imputed-items?" data-toc-modified-id="How-many-imputed-items?-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>How many imputed items?</a></span></li><li><span><a href="#Time-series-plots" data-toc-modified-id="Time-series-plots-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Time series plots</a></span><ul class="toc-item"><li><span><a href="#Behaviors" data-toc-modified-id="Behaviors-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Behaviors</a></span></li><li><span><a href="#Nutrients" data-toc-modified-id="Nutrients-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Nutrients</a></span></li></ul></li></ul></li><li><span><a href="#TWFE-estimator" data-toc-modified-id="TWFE-estimator-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TWFE estimator</a></span><ul class="toc-item"><li><span><a href="#Combined" data-toc-modified-id="Combined-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Combined</a></span></li><li><span><a href="#Just-paper-checks" data-toc-modified-id="Just-paper-checks-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Just paper checks</a></span></li><li><span><a href="#Just-direct-deposit" data-toc-modified-id="Just-direct-deposit-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Just direct deposit</a></span></li><li><span><a href="#Interactions" data-toc-modified-id="Interactions-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Interactions</a></span><ul class="toc-item"><li><span><a href="#All" data-toc-modified-id="All-3.4.1"><span class="toc-item-num">3.4.1&nbsp;&nbsp;</span>All</a></span></li><li><span><a href="#Direct-Deposit" data-toc-modified-id="Direct-Deposit-3.4.2"><span class="toc-item-num">3.4.2&nbsp;&nbsp;</span>Direct Deposit</a></span></li><li><span><a href="#Paper-checks" data-toc-modified-id="Paper-checks-3.4.3"><span class="toc-item-num">3.4.3&nbsp;&nbsp;</span>Paper checks</a></span></li></ul></li></ul></li><li><span><a href="#Sun-and-Abraham" data-toc-modified-id="Sun-and-Abraham-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sun and Abraham</a></span></li><li><span><a href="#Chaisemartin-and-D'Haultfoeuille" data-toc-modified-id="Chaisemartin-and-D'Haultfoeuille-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Chaisemartin and D'Haultfoeuille</a></span></li></ul></div>

# 2008 ESA Analysis
By Zachary A. Goodman

Updated: 2021-05

This script performs the analysis in Goodman (2021).

## Init

In [ ]:
clear all

* set working directory
qui cd "../"

* set global directories
global raw_data ./data/raw_data
global temp_data ./data/temp_data
global gen_data ./data/gen_data
global figures ./tex/figures 
global tables ./tex/tables

* check dependencies
which eventstudyinteract
which reghdfe

* plot style
set scheme s1color

### Helper functions

In [ ]:
capture program drop get_data
program get_data
    use $gen_data/hh_year_week_panel, clear
    end

In [ ]:
capture program drop merge_dems
program merge_dems
    merge m:1 household_code using $gen_data/panelist_info.dta, keep(3) nogen
    sort household_code
    qui by household_code: ereplace wic_indicator_current = min(wic_indicator_current)
    qui by household_code: ereplace wic_indicator_ever_not_current = min(wic_indicator_ever_not_current)
    qui replace wic_indicator_current = 0 if mi(wic_indicator_current)
    qui replace wic_indicator_ever_not_current = 0 if mi(wic_indicator_ever_not_current)
    end

In [ ]:
capture program drop export_img
program export_img
    args name
    graph export $figures/`name'.pdf, name(`name') replace
    end

In [ ]:
* Sample-shrinking function for debugging

capture program drop keep_rand
program keep_rand
    args sample_size  // percent

    capture drop rnum
    bys household_code: gen rnum = runiform() if _n == 1
    by household_code: ereplace rnum = min(rnum)
    keep if rnum < `sample_size'
    end

In [ ]:
* Two-way Fixed Effect estimator

capture program drop twfe
program define twfe
    args outcome leads fname interaction
    /*
    Two-way fixed effect estimator. 
    Must assume homogenous treatment effects and construct a 'never treated' group
    using the last treated cohort.
    
    Args
    outcome: y var
    leads: number of leads to include (lags limited by control group)
    fname: file name
    interaction: optional interaction var
*/

    capture restore
    preserve

    // keep_rand 0.3 // for shrinking sample for debugging
    
    di `leads'

    * drop obs after week of last treatment
    qui sum week_treat
    local max_week = r(max)
    drop if week >= `max_week'
    //tab week_diff if week_treat == `max_week'  // verify last group has no treated obs

    * drop obs before `leads'
    drop if week_diff < -1 * `leads'

    * outcome in logs
    capture drop logy
    gen logy = asinh(`outcome')

    * add week-by-method (to be absorbed)
    * week is 4 digit number, so add 10,000 for check, 20,000 for dd
    replace week = week + hh_method_paper * 10000 + hh_method_dd * 20000

    * factor-ize week_diff
    * leads (n = negative)
    capture drop g_*
    capture drop keepvar
    gen keepvar = 0
    forvalues k = `leads'(-1)1 {
        qui gen g_n`k' = week_diff == -`k'
        qui replace keepvar = 1 if week_diff == -`k'
    }
    * lags
    qui sum week_diff
    local max_week = r(max)
    di `max_week'
    forvalues k = 0/`max_week' {
        qui gen g_`k' = week_diff == `k'
        qui replace keepvar = 1 if week_diff == `k'
    }
    
    * add dummies for interaction model, if necessary
    if "`interaction'" != "" {
        di "Interaction var: `interaction'"
        foreach var of varlist g_* {
            qui gen `var'_inter = `var' * `interaction'
        }
    } 

    * drop reference categories and observations outside experiment
    drop g_n2 g_n`leads'*
    keep if keepvar

    di "Running reghdfe..."

    * standard model, potentially with interactions
    reghdfe logy g_* [fw=projection_factor], ///
    absorb(household_code week) vce(cluster household_code) nocons

    //coefplot , vertical

    * get matrix of coef and se estimates
    matrix C = e(b)  // point estimates in relative time
    mata st_matrix("A", sqrt(st_matrix("e(V)")))  // variance estimates
    matrix A = vecdiag(A)
    matrix C = C \ A  // add row with standard errors
    matrix rownames C = coeff se
    matrix CT = C'  // transpose

    * export matrix
    mat2txt, matrix(CT) saving($temp_data/twfe/`fname'.txt) replace

    end

In [ ]:
* Sun and Abraham estimator

capture program drop sun_abraham
program define sun_abraham
    args outcome weeks fname
    /*
    outcome: y var
    weeks: number of leads and lags
    fname: file name
    */

    capture restore
    preserve

    di `weeks'

    // keep_rand  0.05  // shrink sample for debugging

    * gen logged outcome var
    capture drop logy
    gen logy = asinh(`outcome')

    * control cohort
    capture drop ctrl_cohort
    gen ctrl_cohort = week_treat == 2524

    * drop obs after week of last treatment (2524)
    drop if week >= 2524
    //tab week_diff if week_treat == 2524  // verify last group has no treated obs

    * generate leads and lags
    capture drop g_* 

    * leads (n = negative)
    forvalues k = `weeks'(-1)1 {
        qui gen g_n`k' = week_diff == -`k'
    }

    * lags
    forvalues k = 0/12 {
        qui gen g_`k' = week_diff == `k'
    }

    * drop reference week
    drop g_n`weeks' g_n2

    /*
    eventstudyinteract {y} {rel_time_list} [if] [in] [weight] ,
                   absorb(absvars) cohort(variable) control_cohort(variable)
                   [options covariates(varlist)]
    */
    di "Running eventstudyinteract..."

    eventstudyinteract logy g_* [fw=projection_factor], ///
        absorb(household_code week) cohort(week_treat) control_cohort(ctrl_cohort) ///
        vce(cluster household_code)


    * export coefficients

    matrix C = e(b_iw)  // point estimates in relative time
    mata st_matrix("A", sqrt(st_matrix("e(V_iw)")))  // variance estimates
    matrix C = C \ A  // add row with standard errors
    matrix rownames C = coeff se
    matrix CT = C'  // transpose
    //matrix list C
    //coefplot matrix(C[1]), se(C[2]) vertical
    mat2txt, matrix(CT) saving($temp_data/sun_abraham/`fname'.txt) replace
    
    end

In [ ]:
help eventstudyinteract

## Descriptive stats

In [ ]:
get_data
merge_dems
gunique household_code
gunique household_code week

In [ ]:
tab week_diff

In [ ]:
sum

### Summary table

In [ ]:
* keep only necessary vars
keep household_code week_diff hh_method_paper-projection_factor ///
    items_deals-coupons_amount_saved calories-total_spent hh_* wic*

* keep only pretreatment obs
keep if week_diff < 0  // only pretreatment obs

* collapse to household level
collapse (mean) hh_method_paper-hh_child, by(household_code)

sum

#### Categorical vars

In [ ]:
tab hh_size hh_child

In [ ]:
tab hh_savings_two_months

In [ ]:
capture program drop label_vars
program label_vars

    * 1 - 6+ HH members
    label var hh_size "Household size, top-coded at 6"

    * Low, middle, high income
    label var hh_income "Household income"
    label define income 1 "<$35K" 2 "$35K - $59,999" ///
        3 "$60K - $99,999" 4 ">$100K", replace
    label values hh_income income

    * (Female) Age < 35, 35 - 49, 50 - 64, 65+
    label var hh_age "Age of the (female) head of household"
    label define age 1 "<35" 2 "35 - 49" ///
        3 "50-64" 4 "65+", replace
    label values hh_age age

    * <HS, HS grad, some college, BA+
    label var hh_educ "Education of the (female) head of household"
    label define educ 1 "< HS" 2 "HS Grad" ///
        3 "Some College" 4 "BA+", replace
    label values hh_educ educ

    * White, Black, Hispanic, Asian
    label var hh_race "Racial identity"
    label define race 1 "White" 2 "Black" 3 "Asian" 4 "Other", replace
    label values hh_race race

    * Kids under 18 y/n
    label var hh_child "Any children < 18"
    label define child 1 "Yes" 0 "No", replace
    label values hh_child child

    * Female <30 hrs, 30 - 34 hrs, >= 35 hrs, not employed (includes retired)
    label var hh_emp_fem "Hours employment/week of female head of HH"
    label define emp 1 "< 30" 2 "30 - 34" 3 ">= 35" 9 "Not employed" 0 "No head of this gender", replace
    label values hh_emp_fem emp

    * Male <30 hrs, 30 - 34 hrs, >= 35 hrs, not employed (includes retired)
    label var hh_emp_male "Hours employment/week of male head of HH"
    label values hh_emp_male emp

    * WIC
    gen wic = wic_indicator_current
    replace wic = 2 if wic_indicator_ever_not_current
    label define wic 0 "No" 1 "Current" 2 "Previously", replace
    label values wic wic
    label var wic "WIC indicator"

    * Liquidity
    label define savings 0 "Illiquid" 1 "Liquid", replace
    label values hh_savings_two_months savings
    label var hh_savings_two_months "At least two months income in liquid assets"

    label var hh_method_paper "ESP by check"
    label var hh_method_dd "ESP by dir. dep."
    label var hh_esp_amount "ESP amount"
    label var trips "Trips"
    label var items_deals "Items with deals"
    label var items_storebrand "Storebrand items"
    label var items_scanned "Scanned items"
    label var items_food "Food items"
    label var items_coupons "Items with coupons"
    label var final_price_paid "Total spent, scanned items"
    label var final_price_paid_food "Total spent, scanned food"
    label var coupons_amount_saved "Coupon value"
    label var total_spent "Total spent"

    label var calories "Calories"
    label var caloriesfromfat "Calories from fat"
    label var carbsgrams "Carbohydrates"
    label var cholesterolgrams "Cholesterol"
    label var fatgrams "Fat"
    label var fibergrams "Fiber"
    label var proteingrams "Protein"
    label var satfatgrams "Saturated fat"
    label var sodiumgrams "Sodium"
    label var sugargrams "Sugar"
    label var transfatgrams "Transfat"

    end

In [ ]:
* summary stats for categorical vars

capture restore
preserve

// label_vars 

* unique households
unique household_code if !hh_savings_two_months
unique household_code if hh_savings_two_months
unique household_code

tabout hh_emp* hh_race hh_income hh_age hh_educ hh_child wic hh_savings_two_months ///
    using "$tables/summary_cat.tex", ///
    replace cells(col) format(2) style(tex) bt cl1(2-4) cl2(2-2 3-3 4-4) ///
    topf("$tables/ref/top.tex") botf("$tables/ref/bot.tex") topstr(14cm) botstr(string) ///
    ptotal(none) npos(row)

restore

#### Numeric vars

In [ ]:
tab hh_savings_two_months

In [ ]:
// label_vars 

eststo clear
qui estpost tabstat ///
    hh_method_paper hh_method_dd hh_esp_amount trips total_spent final_price_paid* ///
    items_scanned items_storebrand items_food ///
    items_deals items_coupons coupons_amount_saved ///
    calories-transfatgrams [fw=projection_factor], listwise ///
    statistics(mean sd) columns(statistics) by(hh_savings_two_months)

local caption = "Summary statistics, numeric variables"
local name = "summary_num"
local label = "table_summary_num"
local note = "Variables are at the household-week level. Nutrient variables are all in grams, except for calories. Finanical variables are all in nominal dollars. Statistics are weighted using each household's projection factor as provided by Nielsen."

esttab using $tables/`name'.tex, cells("mean(fmt(2)) sd") nomtitle nonumber unstack label booktabs replace ///
    prehead("\begin{spacing}{1.0} \begin{table} \centering \caption{`caption'} \label{`label'} \begin{threeparttable} \begin{tabular}{m{0.30\linewidth}*{6}{>{\centering\arraybackslash}m{0.09\linewidth}}} \toprule") ///
    postfoot("\bottomrule \end{tabular} \Fignote{`note'} \end{threeparttable} \end{table} \end{spacing}" )

### How many imputed items?

In [ ]:
get_data

capture drop all*
gen all_items = imputed0 + imputed1 + imputed2 + imputed3 + imputed4 + imputed5
egen all_items_scanned = sum(all_items)
// egen all_items_scanned_2 = sum(items_food)
// sum all_items*

forvalues i = 0/5 {
    egen all_imputed_`i' = sum(imputed`i')
    di all_imputed_`i' / all_items_scanned
}

### Time series plots

In [ ]:
get_data
merge_dems

In [ ]:
capture program drop plot_ts
program plot_ts 
    args y byvar title xtitle ytitle

    keep if inrange(week_diff, -12, 11)

    twoway (line `y' week_diff if `byvar' == 1) ///
        (line `y' week_diff if `byvar' == 0), ///
        title("`title'") ytitle("`ytitle'") xtitle("`xtitle'") ///
        legend(off) name("`y'", replace)

    end

#### Behaviors

In [ ]:
capture restore
preserve

local byvar = "hh_savings_two_months"

capture drop week_diff
gen week_diff = week - week_treat
keep items_* total_* final_* coupon `byvar' week_diff projection_factor
collapse (mean) items_* total_* final_* coupon [fw=projection_factor], by(week_diff `byvar')


plot_ts total_spent "`byvar'" "Total spent" "" "Dollars"
plot_ts final_price_paid "`byvar'" "Total spent, scanned items" "" "Dollars"
plot_ts final_price_paid_food "`byvar'" "Total spent, scanned food" "" "Dollars"
plot_ts items_scanned "`byvar'" "Items scanned" "" "Items"
plot_ts items_storebrand "`byvar'" "Storebrand items scanned" "" "Items"
plot_ts items_food "`byvar'" "Food items scanned" "" "Items"
plot_ts items_deals "`byvar'" "Items scanned with deals" "" "Items"
plot_ts items_coupons "`byvar'" "Items scanned with coupons" "" "Items"
plot_ts coupons_amount_saved "`byvar'" "Total coupon value" "" "Dollars"

graph combine total_spent final_price_paid final_price_paid_food ///
    items_scanned items_storebrand items_food ///
    items_deals items_coupons coupons_amount_saved, ///
    row(3) name(raw_behaviors_week, replace) altshrink iscale(1.2) ysize(5) xsize(6)

export_img raw_behaviors_week

restore

#### Nutrients

In [ ]:
capture restore
preserve

local yvars = "calories fatgrams satfatgrams cholesterolgrams carbsgrams sugargrams fibergrams sodiumgrams proteingrams"
local byvar = "hh_savings_two_months"

capture drop week_diff
gen week_diff = week - week_treat
keep `yvars' `byvar' week_diff projection_factor
collapse (mean) `yvars' [fw=projection_factor], by(week_diff `byvar')

plot_ts calories "`byvar'" "Calories" "" "Calories"
plot_ts fatgrams "`byvar'" "Fat" "" "Grams"
plot_ts satfatgrams "`byvar'" "Saturated Fat" "" "Grams"
plot_ts carbsgrams "`byvar'" "Carbohydrates" "" "Grams"
plot_ts sugargrams "`byvar'" "Sugar" "" "Grams"
plot_ts fibergrams "`byvar'" "Fiber" "" "Grams"
plot_ts proteingrams "`byvar'" "Protein" "" "Grams"
plot_ts sodiumgrams "`byvar'" "Sodium" "" "Grams"
plot_ts oz_alcohol "`byvar'" "Alcohol" "" "Ounces"

graph combine calories fatgrams satfatgrams ///
    carbsgrams sugargrams fibergrams ///
    proteingrams sodiumgrams cholesterolgrams, ///
    row(3) name(raw_nutrients_week, replace) altshrink iscale(1.2) ysize(5) xsize(6)

export_img raw_nutrients_week

restore

## TWFE estimator

In [ ]:
get_data
merge_dems

In [ ]:
tab week_treat if hh_method_dd
tab week_treat if hh_method_paper

### Combined

In [ ]:
get_data
desc

In [ ]:
foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    twfe `v' 15 "all/`v'"
}

### Just paper checks

In [ ]:
preserve
keep if hh_method_paper

foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    twfe `v' 15 "paper/`v'"
}

restore

### Just direct deposit

In [ ]:
preserve
keep if hh_method_dd

foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    twfe `v' 15 "dd/`v'"
}

restore

### Interactions

In [ ]:
get_data
merge_dems
gen liquidity_constrained = 1 - hh_savings_two_months

#### All

In [ ]:
foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    twfe `v' 15 "all/interact_`v'" liquidity_constrained
}

#### Direct Deposit

In [ ]:
capture restore
preserve
keep if hh_method_dd

foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    twfe `v' 15 "dd/interact_`v'" liquidity_constrained
}

restore

#### Paper checks

In [ ]:
capture restore
preserve
keep if hh_method_paper

foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    twfe `v' 15 "paper/interact_`v'" liquidity_constrained
}

restore

## Sun and Abraham

In [ ]:
get_data

In [ ]:
* loop through outcomes and get coeffs

foreach v of varlist items_deals-coupons_amount_saved calories-total_spent {
    di "`v'"
    sun_abraham `v' 26 "all/`v'"
}

## Chaisemartin and D'Haultfoeuille

In [ ]:
* Try Chaisemartin & D'Haultfoeuille (2020)
* Very slow

capture restore
preserve

capture drop logx treat
gen logx = asinh(sugargrams)
gen treat = week_diff >= 0

* shrink sample to make sure program works
keep if week_diff >= -52
keep if week_diff <= 52
// gen randn = runiform()
// bys household_code: ereplace randn = max(randn)
// sort randn
// keep in 1/30000

* syntax: outcome group time_period_var treatment_var, options
did_multiplegt logx household_code week treat, switchers(in) breps(100) weight(projection_factor)

restore